In [12]:
import pandas as pd
from myUtils import pickle_load, symb_perf_stats_vectorized

file_close = 'df_close_clean'
path_dir = "C:/Users/ping/MyDrive/stocks/yfinance"
path_data_dump = path_dir + "/VSCode_dump/"

df_c = pickle_load(path_data_dump, file_close)
df_c = df_c[-252::]
format1 = 'df_c({}):\n{}\n{}'.format(len(df_c), df_c.head(3), df_c.tail(3))
# print(format1)

In [13]:
def perf_eval(df_close):
  '''
  df_close is a dataframe with date index, columns of symbols' closing price, and symbol as column name 
  '''
  # %%timeit
  symbols, period_yr, drawdown, UI, max_drawdown, returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI = \
      symb_perf_stats_vectorized(df_close)
  caches_perf_stats_vect = []
  for symbol in symbols:
      date_first = drawdown.index[0].strftime('%Y-%m-%d')
      date_last = drawdown.index[-1].strftime('%Y-%m-%d')
      cache = (symbol, date_first, date_last, period_yr, CAGR[symbol],
              UI[symbol], Std_UI[symbol], CAGR_Std[symbol], CAGR_UI[symbol])
      # append performance data (tuple) to caches_perf_stats (list)
      caches_perf_stats_vect.append(cache)
  column_names = ['symbol', 'first date', 'last date', 'Year', 'CAGR',
                  'UI', 'Std/UI', 'CAGR/Std', 'CAGR/UI']
  # write symbols' performance stats to dataframe
  df_perf = pd.DataFrame(caches_perf_stats_vect, columns=column_names)

  _cols = ['CAGR', 'UI', 'Std/UI', 'CAGR/Std', 'CAGR/UI']
  # print(f'df_perf mean:\n{df_perf[_cols].mean()}\n')
  # print(f'df_perf std:\n{df_perf[_cols].std()}')
  grp_CAGRUI_mean = df_perf['CAGR/UI'].mean()
  grp_CAGRUI_std = df_perf['CAGR/UI'].std()
  grp_CAGRUI_mean_std = grp_CAGRUI_mean / grp_CAGRUI_std
  grp_CAGRUI_mean_std

  grp_CAGRStd_mean = df_perf['CAGR/Std'].mean()
  grp_CAGRStd_std = df_perf['CAGR/Std'].std()
  grp_CAGRStd_mean_std = grp_CAGRStd_mean / grp_CAGRStd_std
  grp_CAGRStd_mean_std

  grp_StdUI_mean = df_perf['Std/UI'].mean()
  grp_StdUI_std = df_perf['Std/UI'].std()
  grp_StdUI_mean_std = grp_StdUI_mean / grp_StdUI_std
  grp_StdUI_mean_std

  return df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std

In [16]:
df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std = perf_eval(df_c)
print(f'grp_StdUI_mean_std :{grp_StdUI_mean_std:.6f}')
print(f'grp_CAGRStd_mean_std :{grp_CAGRStd_mean_std:.6f}')
print(f'grp_CAGRUI_mean_std :{grp_CAGRUI_mean_std:.6f}')
df_perf.head()

grp_StdUI_mean_std :2.133987
grp_CAGRStd_mean_std :-0.323876
grp_CAGRUI_mean_std :0.042199


,symbol,first date,last date,Year,CAGR,UI,Std/UI,CAGR/Std,CAGR/UI
0,A,2021-11-18,2022-11-17,1.0,-0.103570,0.198045,0.110136,-4.748342,-0.522963
1,AA,2021-11-18,2022-11-17,1.0,0.000434,0.374292,0.111396,0.010417,0.001160
2,AAL,2021-11-18,2022-11-17,1.0,-0.280557,0.238188,0.152731,-7.712129,-1.177881
3,AAON,2021-11-18,2022-11-17,1.0,0.043509,0.292579,0.083615,1.778490,0.148708
4,AAP,2021-11-18,2022-11-17,1.0,-0.341663,0.186631,0.127861,-14.317808,-1.830683


In [19]:
# _cols2 = ['TSBK', 'ELF', 'AVEO', 'ASC', 'TPL', 'AXON', 'NRIM', 'AAON', 'UFPT', 'GILD', 'MOD', 'SANM', 'INSW', 'LOPE', 'AE', 'ABMD', 'RMBS', 'WNC']
_cols2 = ['AE', 'ABMD', 'RMBS', 'WNC']
df_temp = df_c[_cols2].copy()
# df_perf, grp_CAGRUI_mean_std = perf_eval(df_temp)
df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std = perf_eval(df_temp)
# print(f'grp_CAGRUI_mean_std :{grp_CAGRUI_mean_std:.6f}')
print(f'grp_StdUI_mean_std: {grp_StdUI_mean_std:.6f}')
print(f'grp_CAGRStd_mean_std: {grp_CAGRStd_mean_std:.6f}')
print(f'grp_CAGRUI_mean_std: {grp_CAGRUI_mean_std:.6f}')
df_perf.head()

grp_StdUI_mean_std: 4.490437
grp_CAGRStd_mean_std: 1.923368
grp_CAGRUI_mean_std: 1.874807


,symbol,first date,last date,Year,CAGR,UI,Std/UI,CAGR/Std,CAGR/UI
0,AE,2021-11-18,2022-11-17,1.0,0.344446,0.137427,0.180161,13.912013,2.506400
1,ABMD,2021-11-18,2022-11-17,1.0,0.107760,0.226766,0.189047,2.513672,0.475203
2,RMBS,2021-11-18,2022-11-17,1.0,0.386102,0.202153,0.129730,14.722480,1.909950
3,WNC,2021-11-18,2022-11-17,1.0,0.367790,0.242663,0.121165,12.508882,1.515644


In [21]:
df_SPY = df_c[['SPY']].copy()
df_SPY
df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std = perf_eval(df_SPY)
print(f'grp_StdUI_mean_std: {grp_StdUI_mean_std:.6f}')
print(f'grp_CAGRStd_mean_std: {grp_CAGRStd_mean_std:.6f}')
print(f'grp_CAGRUI_mean_std: {grp_CAGRUI_mean_std:.6f}')
df_perf.head()

grp_StdUI_mean_std: nan
grp_CAGRStd_mean_std: nan
grp_CAGRUI_mean_std: nan


,symbol,first date,last date,Year,CAGR,UI,Std/UI,CAGR/Std,CAGR/UI
0,SPY,2021-11-18,2022-11-17,1.0,-0.148009,0.139184,0.109105,-9.746598,-1.063406
